<a href="https://colab.research.google.com/github/jackdaus/egolifter/blob/colab/1colab-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Data Access Link

Before you can download ADT data, you need to request from Aria the `ADT_download_urls.json`. This is an insant process. Follow the [directions in the original egolifter github](https://github.com/facebookresearch/egolifter?tab=readme-ov-file#download-and-pre-processing).

# Install Dependencies

In [1]:
# Install uv package manager
!pip install uv

# Verify install
!uv --version

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 118.9 MB/s eta 0:00:00
uv 0.6.16


In [2]:
# Clone the repo (jackdaus version has uv support)
!git clone https://github.com/jackdaus/egolifter.git
%cd egolifter
# Checkout the colab branch. I'm developing compatibility with colab on that branch.
!git checkout colab

Cloning into 'egolifter'...
remote: Enumerating objects: 242, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 242 (delta 82), reused 182 (delta 51), pack-reused 17 (from 1)
Receiving objects: 100% (242/242), 4.76 MiB | 5.17 MiB/s, done.
Resolving deltas: 100% (83/83), done.
/content/egolifter
Branch 'colab' set up to track remote branch 'colab' from 'origin'.
Switched to a new branch 'colab'


In [3]:
# Install packages. This might take a couple of minutes.
!uv sync

Streaming output truncated to the last 5000 lines.
nvidia-cublas-cu12 ------------------------------ 77.10 MiB/346.60 MiB
open3d     ------------------------------ 78.80 MiB/426.94 MiB
nvidia-cudnn-cu12 ------------------------------ 77.09 MiB/633.96 MiB
⠙ Preparing packages... (229/239)
plotly     ------------------------------ 8.28 MiB/14.12 MiB
nvidia-cusolver-cu12 ------------------------------ 77.28 MiB/122.01 MiB
nvidia-nccl-cu12 ------------------------------ 78.39 MiB/179.91 MiB
nvidia-cusparse-cu12 ------------------------------ 77.20 MiB/197.84 MiB
triton     ------------------------------ 78.40 MiB/199.76 MiB
nvidia-cufft-cu12 ------------------------------ 77.81 MiB/201.66 MiB
nvidia-cublas-cu12 ------------------------------ 77.46 MiB/346.60 MiB
open3d     ------------------------------ 79.12 MiB/426.94 MiB
nvidia-cudnn-cu12 ------------------------------ 77.37 MiB/633.96 MiB
⠙ Preparing packages... (229/239)
plotly     ------------------------------ 8.28 MiB/14.12 MiB
nvi

In [5]:
# Note that in order to use the uv virtual environment, we must activate it within
# the cell. (Colab was giving me trouble creating a uv based kernel...)

# To illustrate, notice that the output of the two commands shows different locations
# of the python exectuable in use.
!source .venv/bin/activate; which python

!which python

/content/egolifter/.venv/bin/python
/usr/local/bin/python


In [6]:
# Unfortunately, we must do something convoluted. We use Jupyter notebook magic
# to create a bash script. We then activate the venv. We then run some python code.
# We should see PyTorch version 2.5.1.
%%bash
source .venv/bin/activate

python - <<'PY'
import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())
PY

PyTorch version: 2.5.1+cu124
Torchvision version: 0.20.1+cu124
CUDA is available: True


In [7]:
# Set environment variables (adpated from setup_env.bash in original egolifter)
import os
os.environ.update({
    "EGOLIFTER_PATH":                 "/content/egolifter",
    "GSA_PATH":                       "/content/egolifter/Grounded-Segment-Anything",
    "SAM_CHECKPOINT_PATH":            "/content/egolifter/Grounded-Segment-Anything/sam_vit_h_4b8939.pth",
    "GROUNDING_DINO_CHECKPOINT_PATH": "/content/egolifter/Grounded-Segment-Anything/groundingdino_swint_ogc.pth",
    "SAM_ENCODER_VERSION":            "vit_h",
    "GROUNDING_DINO_CONFIG_PATH":     "/content/egolifter/Grounded-Segment-Anything/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py",
    "EFFICIENTSAM_PATH":              "/content/egolifter/Grounded-Segment-Anything/EfficientSAM",
    "TAG2TEXT_PATH":                  "/content/egolifter/Grounded-Segment-Anything/Tag2Text",
    "ADT_DATA_ROOT":                  "adt",
    "ADT_PROCESSED_ROOT":             "adt_processed",
    "AM_I_DOCKER":                    "False",
    "BUILD_WITH_CUDA":                "True",
    "TAG2TEXT_CHECKPOINT_PATH":       "/content/egolifter/Grounded-Segment-Anything/Tag2Text/tag2text_swin_14m.pth",
    "RAM_CHECKPOINT_PATH":            "/content/egolifter/Grounded-Segment-Anything/Tag2Text/ram_swin_large_14m.pth",
})

In [9]:
# Set up Grounded-Segment-Anything
!git clone https://github.com/IDEA-Research/Grounded-Segment-Anything.git

Cloning into 'Grounded-Segment-Anything'...
remote: Enumerating objects: 1807, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 1807 (delta 9), reused 1 (delta 1), pack-reused 1789 (from 2)
Receiving objects: 100% (1807/1807), 155.84 MiB | 13.18 MiB/s, done.
Resolving deltas: 100% (830/830), done.


In [10]:
# Install SAM
!uv add Grounded-Segment-Anything/segment_anything

Resolved 247 packages in 13.51s
   Building segment-anything @ file:///content/egolifter/Grounded-Segment-Anything/s
   Building segment-anything @ file:///content/egolifter/Grounded-Segment-Anything/s
   Building segment-anything @ file:///content/egolifter/Grounded-Segment-Anything/s
   Building segment-anything @ file:///content/egolifter/Grounded-Segment-Anything/s
      Built segment-anything @ file:///content/egolifter/Grounded-Segment-Anything/s
Prepared 1 package in 613ms
Installed 1 package in 1ms
 + segment-anything==1.0 (from file:///content/egolifter/Grounded-Segment-Anything/segment_anything)


In [47]:
!uv pip install numpy ninja

Using Python 3.11.12 environment at: /usr
Resolved 2 packages in 78ms
Installed 1 package in 3ms
 + ninja==1.11.1.4


In [11]:
%%bash
source .venv/bin/activate
python - <<'PY'
import torch
# import groundingdino
print("torch", torch.__version__, "cuda", torch.version.cuda)
# print("GroundingDINO import OK:", groundingdino.__version__)
PY


torch 2.5.1+cu124 cuda 12.4


In [75]:
%%bash
source .venv/bin/activate
uv pip uninstall torch torchvision torchaudio

Using Python 3.11.12 environment at: /usr


In [12]:
# Make quick-and-dirty fix for issue with GroundingDino not working in Colab
# (due to an early 2025 up stream dependency change).
# See: https://github.com/IDEA-Research/Grounded-Segment-Anything/issues/550
%cd /content/egolifter/Grounded-Segment-Anything/GroundingDINO/groundingdino/models/GroundingDINO/csrc/MsDeformAttn
!sed -i 's/value.type()/value.scalar_type()/g' ms_deform_attn_cuda.cu
!sed -i 's/value.scalar_type().is_cuda()/value.is_cuda()/g' ms_deform_attn_cuda.cu

/content/egolifter/Grounded-Segment-Anything/GroundingDINO/groundingdino/models/GroundingDINO/csrc/MsDeformAttn


In [13]:
# Change back to project directory
%cd ~/../content/egolifter

/content
/content/egolifter


In [14]:
!uv add Grounded-Segment-Anything/GroundingDINO
# uv pip install --upgrade "diffusers[torch]"

Resolved 253 packages in 1.21s
   Building groundingdino @ file:///content/egolifter/Grounded-Segment-Anything/Grou
   Building groundingdino @ file:///content/egolifter/Grounded-Segment-Anything/Grou
⠙ Preparing packages... (0/6)
   Building groundingdino @ file:///content/egolifter/Grounded-Segment-Anything/Grou
⠙ Preparing packages... (0/6)
yapf       ------------------------------     0 B/250.15 KiB
   Building groundingdino @ file:///content/egolifter/Grounded-Segment-Anything/Grou
⠙ Preparing packages... (0/6)
yapf       ------------------------------     0 B/250.15 KiB
tokenizers ------------------------------     0 B/2.90 MiB
   Building groundingdino @ file:///content/egolifter/Grounded-Segment-Anything/Grou
⠙ Preparing packages... (0/6)
yapf       ------------------------------     0 B/250.15 KiB
tokenizers ------------------------------ 16.00 KiB/2.90 MiB
   Building groundingdino @ file:///content/egolifter/Grounded-Segment-Anything/Grou
⠙ Preparing packages... (0/6)
yapf  

In [ ]:
# TODO is this needed?
# !uv pip install --upgrade "diffusers[torch]"

In [15]:
# Download model weights
%cd Grounded-Segment-Anything/
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
!wget https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth
%cd ..

/content/egolifter/Grounded-Segment-Anything
--2025-04-23 13:48:44--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.254.124, 108.157.254.15, 108.157.254.121, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.254.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G   168MB/s    in 14s     

2025-04-23 13:48:58 (181 MB/s) - ‘sam_vit_h_4b8939.pth’ saved [2564550879/2564550879]

--2025-04-23 13:48:58--  https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.co

# Download Data

First, upload your `ADT_download_urls.json` file. Then, run the code below.

## Prepare to download data

In [22]:
# IMPORTANT: Make sure you have uploaded your file `ADT_download_urls.json`!
# See original repo for instructions on how to get that.

file_path = "/content/egolifter/ADT_download_urls.json"

if os.path.exists(file_path):
  print("File found:", file_path)
else:
  print("Error: File ADT_download_urls.json not found!", file_path)
  print("Please upload the ADT_download_urls.json file to the /content/egolifter directory.")
  print("See the original EgoLifter repository for how to get this download link.")
  raise FileNotFoundError("ADT_download_urls.json not found")

File found: /content/egolifter/ADT_download_urls.json


In [26]:
# Sanity check to see if environment variables make their way into the uv venv
%%bash
source .venv/bin/activate

python - <<'PY'
import os
print({os.environ['ADT_DATA_ROOT']})
PY

{'adt'}


In [25]:
%%bash
source .venv/bin/activate

python - <<'PY'

import os
import shutil

# Define data directories
ADT_DATA_ROOT = os.environ['ADT_DATA_ROOT']
ADT_PROCESSED_ROOT = os.environ['ADT_PROCESSED_ROOT']

# Create directories if they don't exist
os.makedirs(ADT_DATA_ROOT, exist_ok=True)
os.makedirs(ADT_PROCESSED_ROOT, exist_ok=True)

# Copy the download URLs JSON file
source_path = "/content/egolifter/ADT_download_urls.json"  # Update if needed
destination_path = os.path.join(ADT_DATA_ROOT, "ADT_download_urls.json")
shutil.copy(source_path, destination_path)

PY

In [27]:
# Copy the vignette images to the dataset
!cp assets/vignette_imx577.png ${ADT_DATA_ROOT} # Vignette image for the RGB camera
!cp assets/vignette_ov7251.png ${ADT_DATA_ROOT} # Vignette image for the SLAM camera

In [28]:
# Move the vignette files
import shutil

# Define the destination paths within the data directory
vignette_rgb_destination = os.path.join(os.environ['ADT_DATA_ROOT'], "vignette_imx577.png")
vignette_slam_destination = os.path.join(os.environ['ADT_DATA_ROOT'], "vignette_ov7251.png")

# Copy the vignette images
shutil.copy("assets/vignette_imx577.png", vignette_rgb_destination)
shutil.copy("assets/vignette_ov7251.png", vignette_slam_destination)

'adt/vignette_ov7251.png'

## Actually download data

In [29]:
# Define the scene names in a Python list. For now, we just have one sample scene.
scene_names_new = ["Apartment_release_golden_skeleton_seq100_10s_sample_M1292"]
scene_names     = ["Apartment_release_golden_skeleton_seq100_10s_sample"]

# Loop through the scene names and execute the download command for each
for scene_name in scene_names_new:
    !uvx --from projectaria-tools aria_dataset_downloader \
        -c adt/ADT_download_urls.json \
        -o adt/ \
        -d 0 1 2 3 6 7 \
        -l {scene_name}

⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
charset-normalizer ------------------------------ 140.49 KiB/140.49 KiB
⠙ Preparing packages... (0/5)
charset-normalizer ------------------------------ 140.49 KiB/140.49 KiB
⠙ Preparing packages... (0/5)
charset-normalizer ------------------------------ 140.49 KiB/140.49 KiB
pillow     ------------------------------     0 B/4.39 MiB
⠙ Preparing packages... (0/5)
charset-normalizer ------------------------------ 140.49 KiB/140.49 KiB
pillow     ------------------------------ 16.00 KiB/4.39 MiB
⠙ Preparing packages... (0/5)
charset-normalizer ------------------------------ 140.49 KiB/140.49 KiB
pillow     ------------------------------ 16.00 KiB/4.39 MiB
projectar

# Process Data (Part 1)

In [30]:
# Process the images to be in a format appropriate for 3dgs
for scene_name in scene_names:
  !uv run python scripts/process_adt_3dgs.py \
    --data_root adt \
    --output_root adt_processed \
    --sequence_name {scene_name}

Processing sequence adt/Apartment_release_golden_skeleton_seq100_10s_sample_M1292
[AriaDigitalTwinDataPathsProvider][WARNING]: no depth image data found at adt/Apartment_release_golden_skeleton_seq100_10s_sample_M1292/depth_images_with_skeleton.vrs
[AriaDigitalTwinDataPathsProvider][WARNING]: no synthetic video data found at adt/Apartment_release_golden_skeleton_seq100_10s_sample_M1292/synthetic_video.vrs
[MpsDataPathsProvider][WARNING]: Hand tracking folder (adt/Apartment_release_golden_skeleton_seq100_10s_sample_M1292/mps/hand_tracking) does not exist in MPS root folder, not loading wrist and palm poses.
[ProgressLogger][INFO]: 2025-04-23 13:58:38: Opening adt/Apartment_release_golden_skeleton_seq100_10s_sample_M1292/video.vrs...
[MultiRecordFileReader][DEBUG]: Opened file 'adt/Apartment_release_golden_skeleton_seq100_10s_sample_M1292/video.vrs' and assigned to reader #0
[VrsDataProvider][INFO]: streamId 211-1/camera-et activated
[VrsDataProvider][INFO]: streamId 214-1/camera-rgb act

# Train on Vanilla 3DGS Pipeline

This is a first test of training on the vanilla 3DGS pipeline.

In [39]:
# Run the vanilla 3dgs pipeline.
# It will ask if you want to log into wandb to visualize training progress/logs
# Run the code. In Colab, we must limit number of worker threads to 2.
!uv run python train_lightning.py \
  scene.scene_name=Apartment_release_golden_skeleton_seq100_10s_sample \
  scene.data_root=$ADT_PROCESSED_ROOT \
  exp_name=3dgs \
  output_root=./output/adt \
  wandb.project=egolifter_adt \
  scene.num_workers=2

Apartment_release_golden_skeleton_seq100_10s_sample
/content/egolifter/.venv/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'train': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
Working directory : /content/egolifter
Output directory  : /content/egolifter/output/adt/Apartment_release_golden_skeleton_seq100_10s_sample/vanilla_3dgs/2025-04-23_14-05-57
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jackdaus (jackdaus-george-mason-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.19.9
wandb: Run data is saved locally in ./output/adt/Apartment_release_golden_skeleton_seq100_10s_sample/vanilla_3dgs/wandb/run-20250423_140557-942afenz
wandb: Run `wandb offline` to turn off s

# Process Data (Part 2)

## Segmentation

In [ ]:
# Generate the SAM segmentation results. This takes about 20 to 30 minutes.
!uv run python scripts/generate_gsa_results.py \
  -i adt_processed/Apartment_release_golden_skeleton_seq100_10s_sample \
  --class_set none \
  --sam_variant sam \
  --max_longer_side 512 \
  --no_clip

Skipping tagging and detection models. 
Found global_points.csv.gz file, assuming Aria data set!
 14% 37/267 [03:06<19:11,  5.01s/it]

## Generate evaluation target for query-based segmentation

### generate_2dseg_query

In [127]:
%%bash
source .venv/bin/activate

SCENE_NAME="Apartment_release_golden_skeleton_seq100_10s_sample"

uv run python scripts/generate_2dseg_query.py \
  --data_root $ADT_PROCESSED_ROOT \
  --scene_name $SCENE_NAME

100%|██████████| 267/267 [04:33<00:00,  1.02s/it]


### generate_2dseg_query_sample

In [129]:
%%bash
source .venv/bin/activate

SCENE_NAME="Apartment_release_golden_skeleton_seq100_10s_sample"

uv run python scripts/generate_2dseg_query_sample.py \
  --data_root $ADT_PROCESSED_ROOT \
  --scene_name $SCENE_NAME

Number of training images: 170
Sampled queries saved to adt_processed/Apartment_release_golden_skeleton_seq100_10s_sample/2dseg_query_sample.json


### generate_3dbox_query

In [130]:
%%bash
source .venv/bin/activate

SCENE_NAME="Apartment_release_golden_skeleton_seq100_10s_sample"

uv run python scripts/generate_3dbox_query.py \
  --raw_root $ADT_DATA_ROOT \
  --data_root $ADT_PROCESSED_ROOT \
  --scene_name $SCENE_NAME

Loaded #closed loop trajectory poses records: 300
Loaded #EyeGazes: 300


[AriaDigitalTwinDataPathsProvider][WARNING]: no depth image data found at adt/Apartment_release_golden_skeleton_seq100_10s_sample_M1292/depth_images.vrs
[AriaDigitalTwinDataPathsProvider][WARNING]: no synthetic video data found at adt/Apartment_release_golden_skeleton_seq100_10s_sample_M1292/synthetic_video.vrs
[MpsDataPathsProvider][WARNING]: Hand tracking folder (adt/Apartment_release_golden_skeleton_seq100_10s_sample_M1292/mps/hand_tracking) does not exist in MPS root folder, not loading wrist and palm poses.
[ProgressLogger][INFO]: 2025-04-23 07:03:31: Opening adt/Apartment_release_golden_skeleton_seq100_10s_sample_M1292/video.vrs...
[MultiRecordFileReader][DEBUG]: Opened file 'adt/Apartment_release_golden_skeleton_seq100_10s_sample_M1292/video.vrs' and assigned to reader #0
[VrsDataProvider][INFO]: streamId 211-1/camera-et activated
[VrsDataProvider][INFO]: streamId 214-1/camera-rgb activated
[VrsDataProvider][INFO]: streamId 247-1/baro0 activated
[VrsDataProvider][WARNING]: Unsup

# EgoLifter (full method)

In [135]:
# EgoLifter (full method). This takes about 5 hours on the current example data.
!uv run python train_lightning.py \
    scene.scene_name="Apartment_release_golden_skeleton_seq100_10s_sample" \
    scene.data_root=$ADT_PROCESSED_ROOT \
    model=unc_2d_unet \
    model.unet_acti=sigmoid \
    model.dim_extra=16 \
    lift.use_contr=True \
    exp_name=egolifter \
    output_root=./output/adt \
    wandb.project=egolifter_adt \
    scene.num_workers=2

/content/egolifter/.venv/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'train': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
Working directory : /content/egolifter
Output directory  : /content/egolifter/output/adt/Apartment_release_golden_skeleton_seq100_10s_sample/unc_2d_unet_egolifter/2025-04-23_07-12-52
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jackdaus (jackdaus-george-mason-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.19.9
wandb: Run data is saved locally in ./output/adt/Apartment_release_golden_skeleton_seq100_10s_sample/unc_2d_unet_egolifter/wandb/run-20250423_071253-vci53wyu
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run egolift